# Word2Vec Skip model


## 1.  Word Embedding
由于 onr-hot 映射的稀疏、词库巨大、不可以表示词的相关性等的缺点，可以使用词嵌入的方式对数据进行降维。一般使用词嵌入具有以下优势：

   + 便于分布式 表示
   + 可以使用连续值进行表示
   + 更低的维度
   + 可以获取词之间的语义关联
    
> ```shell
Representing a word by means of its neighbors
“You shall know a word by the company it keeps.”
                                                                -- Firth, J. R. 1957:11
```

## 2. Word2Vec
### 2.1 概述

Word2Vec 使用了一个典型的机器学习中的降维的技巧，通过训练一个具有单隐含层的的神经网络。然后通过学习所得隐含层的权重作为实际的词向量，实现数据将为的目的。

> 在非监督学习中，Auto-encoder 也使用了这样的技巧。可以使用自动编码机将输入向量进行压缩；使其在隐含层降维；在输出层可以将其解压缩到原来的向量，训练结束之后可以剥离输出层，使用隐含层。可以通过这种方式学习良好的图像特征而无需标记训练数据的技巧。

word2vec 具有两个模型，CBOW and skip-gram. 两个模型原理相似，唯一不同之处在于，连续词包模型是根据上下文预测一个词，而跳表模型是根据一个中心词预测其上下文。从数据分析上说， CBOW 模型具有平滑大量分布信息的效果（将上下文视为一个观察），通常对于小数据集是有效的。但是 skip-gram 将每一个 `context-target` 视为一个观察，通常在大数据集上的表现更好。

### 2.2 skip-gram 基本概念
需要构建一个神经网络去完成这样一个任务：给定一个句子的中心词，随机查找并挑选其附近的词。神经网络需要告诉我们词库中的每一个词出现在中心词附近的概率。

附近的词需要使用**窗口（window）**的概念描述：
 > window size: 需要考虑中心词左边w，以及右边w个词；一般不大于5



![skip-gram](http://mccormickml.com/assets/word2vec/training_data.png)

### 2.3 skip-gram 细节
首先需要创建一个词库，使用 one-hot 编码，对于一个大小为N的词库而言，每一个词都是N维向量。作为模型的输入。模型的输出也是一个 N维 向量，包含了每一个词出现在中心词 **附近** 的概率。基本的结构如下：
![arch](http://mccormickml.com/assets/word2vec/skip_gram_net_arch.png)

以上模型没有使用非线性激活函数， `Output Layer` 使用了softmax，用以表示所有词与中心词的相关性。训练以上模型时，输入是一个 one-hot vector, 输出是一个概率分布函数。

在以上模型中，如果需要训练的词向量有300个特征，或者说将原有的 N-d vector 映射到一个 300 维的向量空间中。那么隐含层的 `shape =[N, 300]` . **隐含层就是我们需要的词向量查找表**

![look](http://mccormickml.com/assets/word2vec/word2vec_weight_matrix_lookup_table.png)


#### Output Layer
输出层使用一个 `softmax regression classfier`, 也就是多元逻辑回归分类器。

## 3. tensorflow 操作

### 3.1 Embedding Lookup Ops

$[0\;0\;0\; 1\; 0]\quad \times \quad  \left[ \begin{matrix}
   17 & 24 & 1 \\
   23 & 5 & 7 \\
   4 & 6 & 13 \\
   \color{green}{10} & \color{green}{12} & \color{green}{19} \\
   11 & 18 & 25
  \end{matrix} \right]  = [10 \;12 \; 19]\tag{1}$
 
 > tf.nn.embedding_lookup(params, ids, partition_strategy='mod', name=None, validate_indices=True, max_norm=None)
 

 
 ### 3.2 NCE Loss function
 关于损失函数的选取，可以使用 softmax ，但是需要大量的计算，可以使用基于取样的方式。
 **负取样（Negative Sampling）** 是一种简化的 **噪声对比预测 NCE（Noise Contrastive Estimation）**。 使用NCE可以近似 softmax，但是使用负取样不可以。[CS224N](http://web.stanford.edu/class/cs224n/) 

> nce_loss(weights, biases, labels, inputs, num_sampled, num_classes, num_true=1, sampled_values=None, remove_accidental_hits=False, partition_strategy='mod', name='nce_loss')







## 4. 构建模型
为了使模型可复用，可以试用一下策略：
+ 为模型定义一个类
+ 在集合类型中建立模型
+ 将模型的 `graph_def` 存储在文件中，需要使用时不需要重建


  

Word2Vec skip-gram with NCE loss function

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.contrib.tensorboard.plugins import projector

import utils
import word2vec_utils

设置模型超参数

In [3]:
VOCAB_SIZE = 50_000
BATCH_SIZE = 128
EMBED_SIZE = 128     # demision of the word embedding
SKIP_WINDOW = 1      # 上下文窗口大小
NUM_SAMPLED = 64  # 负取样的大小
lr = 1.0
NUM_TRAIN_STEP = 100_000
VISUAL_FLD = 'visualization'
SKIP_STEP = 5_000

需要下载的数据：

In [4]:
DOWNLOAD_URL = 'http://mattmahoney.net/dc/text8.zip'
EXPECTED_BYTES = 31344016
NUM_VISUALIZE = 3000        # number of tokens to visualize

使用函数式定义模型：

In [5]:
def word2vec(dataset):
    """
    dataset: The input, output tf.data.Dataset
    """
    # get input ,output  from dataset
    with tf.name_scope('data'):
        iterator = dataset.make_initializable_iterator()
        center_words, target_words = iterator.get_next()
        
    # Define weights and embedding look up
    with tf.name_scope('embedd'):
        embed_matrix = tf.get_variable('embed_matrix',
                                      shape=[VOCAB_SIZE, EMBED_SIZE],
                                      initializer=tf.random_uniform_initializer())
        embed = tf.nn.embedding_lookup(embed_matrix, center_words, name="embedding")
    
    # Construct variables for NCE loss and define loss function
    with tf.name_scope('loss'):
        nce_weight = tf.get_variable('nce_weight',
                                    shape=[VOCAB_SIZE, EMBED_SIZE],
                                    initializer=tf.truncated_normal_initializer(stddev=1.0 / (EMBED_SIZE ** 0.5)))
        nce_bias = tf.get_variable('nce_bias', 
                                  initializer=tf.zeros([VOCAB_SIZE]))
        
        #define loss 
        loss = tf.reduce_mean(tf.nn.nce_loss(
            weights=nce_weight,
            biases=nce_bias,
            inputs=embed,
            labels=target_words,
            num_sampled=NUM_SAMPLED,
            num_classes=VOCAB_SIZE), name='loss')
        
    with tf.name_scope('optimizer'):
        optimizer =tf.train.GradientDescentOptimizer(lr).minimize(loss)
        
    utils.safe_mkdir('checkpoints')
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        sess.run(iterator.initializer)
        
        total_loss = 0.0 
        writer = tf.summary.FileWriter('graphs/word2vec_simple', sess.graph)
        
        for i in range(NUM_TRAIN_STEP):
            try:
                loss_batch, _ = sess.run([loss, optimizer])
                total_loss += loss_batch
                if (i + 1) % SKIP_STEP == 0:
                    print(f'Average loss at step {i}: {total_loss / SKIP_STEP:5.1f}')
                    total_loss = 0
            except tf.errors.OutOfRangeError:
                sess.run(iterator.initializer)
        writer.close()
        
        
        
        

从生成器中获取数据：

In [6]:
def gen():
    yield from word2vec_utils.batch_gen(DOWNLOAD_URL, EXPECTED_BYTES, VOCAB_SIZE, 
                                        BATCH_SIZE, SKIP_WINDOW, VISUAL_FLD)

In [7]:
def main():
    dataset = tf.data.Dataset.from_generator(gen, 
                                (tf.int32, tf.int32), 
                                (tf.TensorShape([BATCH_SIZE]), tf.TensorShape([BATCH_SIZE, 1])))
    word2vec(dataset)

In [8]:
if __name__ == '__main__':
    main()

Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, use
    tf.py_function, which takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Successfully downloaded data/text8.zip
Average loss at step 4999:  65.1
Average loss at step 9999:  18.5
Average loss at step 14999:   9.6
Average loss at step 19999:   6.7
Average loss at step 24999:   5.7
Average loss at step 29999:   5.2
Average loss at step 34999:   5.0
Average loss at step 39999:   4.8
Average loss at step 44999:   4.8
Average loss at step 49999:   4.8
Average loss at step 54999:   4.7
Average loss at step 59999:   4.7
A